In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
# !pip install transformers==3.0.2
# !pip install torch

  Using cached mxnet-1.9.0-py3-none-manylinux2014_x86_64.whl (47.3 MB)
  Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.19.1
    Uninstalling graphviz-0.19.1:
      Successfully uninstalled graphviz-0.19.1
Processing /home/adminuser/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c/gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl


In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-dn7eyhj_
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-dn7eyhj_
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15662 sha256=e177ec68e15f33354c4a82bc42757bea1f5bff540dfd20ba74a7fd858beca4a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-ad61mwkd/wheels/bf/5f/74/81bf3a1332130eb6629ecf58876a8746b77021e7d7b0638e91
Successfully built kobert


In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [2]:
import pandas as pd
data = pd.read_csv('./total_final_새로운별점_0제거.csv', encoding = 'utf-8', index_col = 0)

In [3]:
data.head()

,Unnamed: 0.1,type,review,star,label,new_label
0,0,금성침대 파인드 LED 서랍형 침대 SS,좋은 가격에 잘 구매했습니다 다만 수납공간은 조금 많이 작네요 이 정도였으면 그냥 ...,3,0,0
2,2,금성침대 파인드 LED 서랍형 침대 SS,별로예요 사진처럼 침대 하단부에 시트지가 조금 벗겨졌어요 기사님 설치하실 때 그런 ...,2,0,0
3,3,금성침대 파인드 LED 서랍형 침대 SS,설치다 하시고 기사님은 가셨고 프레임 헤드 쪽이 깨져 있었어요 못 보신 건지 말씀 ...,2,0,0
4,4,금성침대 파인드 LED 서랍형 침대 SS,월일에 설치해서 딱 두 달 사용했는데 서랍장이 분리되어 버렸습니다 서랍장이 톱밥 뭉...,1,0,0
5,5,금성침대 파인드 LED 서랍형 침대 SS,보통 이에 요 넉넉한 수납공간이라 해서 구매했는데 서랍은 서랍 자체 밑바닥에 바퀴가...,3,0,0


In [4]:
data = data.drop(['Unnamed: 0.1', 'star', 'label'], axis=1)

In [5]:
data = data.drop(['type'], axis = 1)

In [6]:
data

,review,new_label
0,좋은 가격에 잘 구매했습니다 다만 수납공간은 조금 많이 작네요 이 정도였으면 그냥 ...,0
2,별로예요 사진처럼 침대 하단부에 시트지가 조금 벗겨졌어요 기사님 설치하실 때 그런 ...,0
3,설치다 하시고 기사님은 가셨고 프레임 헤드 쪽이 깨져 있었어요 못 보신 건지 말씀 ...,0
4,월일에 설치해서 딱 두 달 사용했는데 서랍장이 분리되어 버렸습니다 서랍장이 톱밥 뭉...,0
5,보통 이에 요 넉넉한 수납공간이라 해서 구매했는데 서랍은 서랍 자체 밑바닥에 바퀴가...,0
...,...,...
21304,폭이 아쉽기는 하지만 튼튼하고 분위기 있어요,1
21305,잘 사용하고 있어요 좋아요,1
21306,튼튼하고 수납공간도 좋아서 여전히 만족해요,1
21307,지방이다 보니 배송이 좀 늦고 기사님이 바쁘셔서 그냥 집에 두고 가시다 보니 균형 ...,0


In [7]:
data_list = []
for q, label in zip(data['review'], data['new_label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [8]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[-1])

['좋은 가격에 잘 구매했습니다 다만 수납공간은 조금 많이 작네요 이 정도였으면 그냥 없는 모델 살 걸 그랬네요 라이트도 불이 단계 조절이 되는 걸로 알고 있었는데 어떻게 하는지 설명서가 없어서 상품 페이지 보고 알았네요 매트리스는 진짜 끝내주네 요 매트는 별 다섯입니다 배송과 설치는 빨리 잘해주셨는데 기다린 시간이 너무 오래 걸리네 요 설치는 잘 받았어요', '0']
['제가 생각했던 것보다 커서 당황했지만 예뻐 요배송 기사분도 친절하시고요', '1']
['여자 혼자 조립하려니 제품 자체가 너무 무거웠어요 ㅠㅠ', '0']
['조립 부속이 잘 안 맞아요 부속 불량도 있고요', '0']
['앉아서 오래 일해서 허리가 아파서 구매했습니다 설치하고 시간 정도 정리한 거 같아 요 너무너무 맘에 들어 요 사이즈인데 키보드 칠 때마다 약간씩 모니터 흔들거림이 있네요 그래도 튼튼한 거 같아', '0']


In [9]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split                                            
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

In [10]:
print(len(dataset_train))
print(len(dataset_test))

15928
3983


In [11]:
dataset_train

[['가격 대비가 성비 좋습니다 후기 보고 걱정했는데 제품 깨끗해요', '1'],
 ['우선 맘에 들어 요 깔끔한 거 좋아하시는 분들은 다 만족하실 거 같고 요 별점인 건 틀이 안 맞아서요', '1'],
 ['구성비 진짜 좋아요 냥이 키우시는 분들 고민 말 구사세요ㅋㅋ 힘센 냥이도 마리 키우는데 발톱 안 들어가요', '1'],
 ['최고 예 요 주정도 기다렸던 것 같아요 그래도 기다린 만큼 보람 있는 물건이에요 잘 사용하겠습니다', '0'],
 ['코로나로 한 달 이상 기다려 배송받았습니다ㅠ 설치 직접 하는 거라 좀 힘들었지만 어렵지는 않았습니다', '0'],
 ['두 개나 구매했는데 먼지 좀 잘 붙지만 그거 감안해서도 너무 좋아요 ㅋㅋ', '0'],
 ['한 달 써보니깐 침대로 사용하기에는 별로인 것 같아요 가운데 높낮이가 너무 심해서 허리 아파요', '0'],
 ['뒤에 멀티탭 정리함 배 선 정리 깔끔한 뒤 단점은 좁아 여 너무 불편해요 더 큰 문제는 개별 스위치 끌 수가 없어요 이게 뭔가 싶어요 견고한데 뭔가 좀 고급 진거 빼곤 개발자들 멀티탭만 넣을 생각했지 개별 스위치 꺼 보진 않았나 봐요 미완성 제품',
  '0'],
 ['침대가 튼튼하고 흔들림이 없는 점 아주 좋아요', '0'],
 ['배송도 빠르고 회사에서 야근용으로 딱이네요', '1'],
 ['튼튼하고 좋아요 많이 파세요', '1'],
 ['빠른 배송과 배송기사님이 조립해주시는 건 좋은데 듀얼 모니터 들어가는 부분에 말도 없이 선정 링 케이스를 달아서 듀얼 모니터가 안 들어가서 케이스 빼달라 했네요 상판에나 사 박은 구멍만 생겼네요 선정 리 케이스 달 때는 말하고 달라주셨으면 합니다',
  '0'],
 ['보이는 대로 그대로입니다 딱 가격만큼 이네요', '1'],
 ['사무실에서 쓸려고 구매했는데 생각보다 좋아요', '1'],
 ['원하던 색상 그대로였고 튼튼하네 요가 성비 최고입니다 재구매 의사 있어요', '1'],
 ['다리가 가늘지만 안정적으로 튼튼하고 상판이 내구성도 좋고 디자인도 마음

In [12]:
device = torch.device("cuda:0")

In [13]:
#bert 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/adminuser/notebooks/안유진/.cache/kobert_v1.zip
using cached model. /home/adminuser/notebooks/안유진/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
# Setting parameters
max_len = 100
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [16]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/adminuser/notebooks/안유진/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [17]:
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [18]:
data_train[0]

(array([   2,  728, 1649, 5330, 2781, 6441, 4204, 6701, 5176, 5561, 2358,
         880, 7867, 4158, 1343, 7848, 6999,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1], dtype=int32),
 array(18, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [20]:
torch.cuda.is_available()

True

In [21]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [22]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [23]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [25]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_3344/2673822008.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/249 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6866124868392944 train acc 0.53125
epoch 1 batch id 201 loss 0.3047824800014496 train acc 0.8002176616915423
epoch 1 train acc 0.8173228628800918


/tmp/ipykernel_3344/2673822008.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/63 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9427083333333334


  0%|          | 0/249 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.004219656810164452 train acc 1.0
epoch 5 batch id 201 loss 0.003329798812046647 train acc 0.9931592039800995
epoch 5 train acc 0.9935993975903614


  0%|          | 0/63 [00:00<?, ?it/s]

epoch 5 test acc 0.9429563492063492


In [26]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("부정")
            elif np.argmax(logits) == 1:
                test_eval.append("긍정")

        print(">> 입력한 리뷰는 " + test_eval[0] + " 입니다.")

using cached model. /home/adminuser/notebooks/안유진/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [27]:
# 무한반복 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("리뷰데이터 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

KeyboardInterrupt: Interrupted by user

In [ ]:
from transformers import pipeline
clf = pipeline('sentiment-analysis')
result = cld("hate")[0]

print('감성분석 결과: %s, 감성스코어: %0.4f' % (result['label'], result['score']))